In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('data/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [6]:
# Create our features
features = df.drop(target, axis=1)
X = pd.get_dummies(features)

# Create our target
y = df[target]


In [7]:
# y[:5]

In [8]:
# X.head()

In [9]:
# X.describe()

In [10]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20, stratify=y)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [12]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
rf = BalancedRandomForestClassifier(n_estimators=100, random_state=20)
rf.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=20)

In [13]:
# Calculated the balanced accuracy score
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = rf.predict(X_test)
rndForest_bas = balanced_accuracy_score(y_test,y_pred)

In [14]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)

rndForest_cm = pd.DataFrame(
    cm, index=["Actual High_Risk", "Actual Low_Risk"], columns=["Predicted High_Risk", "Predicted Low_Risk"])

rndForest_cm

,Predicted High_Risk,Predicted Low_Risk
Actual High_Risk,51,36
Actual Low_Risk,1801,15317


In [15]:
# Print the imbalanced classification report

rndForest_ICM = classification_report_imbalanced(y_test, y_pred)
print(rndForest_ICM)


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.59      0.89      0.05      0.72      0.51        87
   low_risk       1.00      0.89      0.59      0.94      0.72      0.54     17118

avg / total       0.99      0.89      0.59      0.94      0.72      0.54     17205



In [16]:
# List the features sorted in descending order by feature importance
feature = []
imp_score = []

importances = sorted(zip(rf.feature_importances_, X.columns), reverse=True)
for importance in importances:
    feature.append(f"{importance[1]}")
    imp_score.append(f"{importance[0]*100:.1f}%")

importance_dict={"Importance":imp_score}
imp_df=pd.DataFrame(importance_dict,index=feature)
imp_df.head(10)


,Importance
total_pymnt_inv,7.9%
total_rec_prncp,6.9%
total_rec_int,6.6%
total_pymnt,5.8%
last_pymnt_amnt,5.6%
int_rate,2.8%
max_bal_bc,2.2%
mo_sin_old_rev_tl_op,1.8%
installment,1.7%
issue_d_Jan-2019,1.6%


### Easy Ensemble AdaBoost Classifier

In [17]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
EE = EasyEnsembleClassifier(n_estimators=100, random_state=20)
EE.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=20)

In [18]:
# Calculated the balanced accuracy score
y_pred = EE.predict(X_test)
eEnsemble_bas = balanced_accuracy_score(y_test,y_pred)

In [19]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)

eEnsemble_cm = pd.DataFrame(
    cm, index=["Actual High_Risk", "Actual Low_Risk"], columns=["Predicted High_Risk", "Predicted Low_Risk"])

eEnsemble_cm

,Predicted High_Risk,Predicted Low_Risk
Actual High_Risk,72,15
Actual Low_Risk,809,16309


In [20]:
# Print the imbalanced classification report
eEnsemble_ICM = classification_report_imbalanced(y_test, y_pred)

# Results for Analysis
---
## Balanced Random Forest Classifier

In [21]:
print(f"Balanced Accuracy Score: {round(rndForest_bas*100,2):.1f}%")

Balanced Accuracy Score: 74.0%


In [22]:
rndForest_cm

,Predicted High_Risk,Predicted Low_Risk
Actual High_Risk,51,36
Actual Low_Risk,1801,15317


In [23]:
print(rndForest_ICM)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.59      0.89      0.05      0.72      0.51        87
   low_risk       1.00      0.89      0.59      0.94      0.72      0.54     17118

avg / total       0.99      0.89      0.59      0.94      0.72      0.54     17205



### Feature Importance, Ranked

In [24]:
imp_df.head(10)

,Importance
total_pymnt_inv,7.9%
total_rec_prncp,6.9%
total_rec_int,6.6%
total_pymnt,5.8%
last_pymnt_amnt,5.6%
int_rate,2.8%
max_bal_bc,2.2%
mo_sin_old_rev_tl_op,1.8%
installment,1.7%
issue_d_Jan-2019,1.6%


---
## Easy Ensemble AdaBoost Classifier

In [25]:
print(f"Balanced Accuracy Score: {round(eEnsemble_bas*100,2):.1f}%")

Balanced Accuracy Score: 89.0%


In [26]:
eEnsemble_cm

,Predicted High_Risk,Predicted Low_Risk
Actual High_Risk,72,15
Actual Low_Risk,809,16309


In [27]:
print(eEnsemble_ICM)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.08      0.83      0.95      0.15      0.89      0.78        87
   low_risk       1.00      0.95      0.83      0.98      0.89      0.80     17118

avg / total       0.99      0.95      0.83      0.97      0.89      0.80     17205

